## Merging of Per-Core Logs
### Can we assume the following about per-core behavior: 
#### If all cores of a running experiment do similar work, then their individual behaviors - as exposed by their per-core logs - are similar to each other, and hence, the behavior of all cores can be merged into an overall behavior of the full experimental run.

In [1]:
import os
import time
import pandas as pd
import numpy as np

In [2]:
import eigen_analysis

cols = eigen_analysis.LINUX_COLS
time_unit = eigen_analysis.TIME_CONVERSION_khz
joules_unit = eigen_analysis.JOULE_CONVERSION

In [3]:
# get all itrs explored for some (dvfs, qps) pair
def list_itrs(rdtsc_dirname):
    itrs = []
    for file in os.listdir(rdtsc_dirname):
        tags = file.split('_')
        itr = tags[1]
        itrs.append(itr)
    itrs = list(set(itrs))
    return itrs

In [4]:
global itrs

In [5]:
def print_err_log(dvfs, qps, itr, rapl='135', run='0', err_dir = 'err_logs/'):
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    if os.path.exists(err_dir + err_filename):
        err_file = open(err_dir + err_filename, 'r')
        print(err_file.read())
    else:
        print('EMPTY ERROR LOG')

In [6]:
def handle_neg_diffs(df_diffs, df, core, err_file):
    
    tmp = df_diffs.copy()

    # isolating rows with negative diffs
    tmp_neg = tmp[(tmp['joules_diff'] < 0) | (tmp['instructions_diff'] < 0) | (tmp['cycles_diff'] < 0) \
                   | (tmp['ref_cycles_diff'] < 0) | (tmp['llc_miss_diff'] < 0) | (tmp['timestamp_diff'] < 0)]

    # re-computing diffs if possible; else dropping rows
    for i,j in tmp_neg.iterrows():
        prev = df.shift(1).loc[i]
        cur = df.loc[i]
        # TODO note that we are only handling case of RAPL-energy-status register overflow
        if (tmp.loc[i]['joules_diff'] < 0) & (tmp.loc[i]['timestamp_diff'] >= 0.001):
            err_file.write('CORE ' + str(core) + '  ---  JOULES COUNTER OVERFLOW AT LOG ENTRY DIFF #' + str(i) + '\n')
            tmp.loc[i, ['joules_diff']] = (2**32 - 1) * joules_unit - prev['joules'] + cur['joules'] 
        else:
            err_file.write('CORE ' + str(core) + '  ---  UNEXPLAINED NEGATIVE VALS AT LOG ENTRY DIFF # ' + str(i) + '\n')
            cols = ''
            prevs = ''
            currs = ''
            for col in list(df.columns):
                cols += col + '  '
                prevs += str(prev[col]) + '  '
                currs += str(cur[col]) + '  '
            err_file.write('          ' + cols + '\n')
            err_file.write('         log[' + str(i-1) + ']: ' + prevs + '\n')
            err_file.write('         log[' + str(i) + ']: ' + currs + '\n')
            tmp = tmp.drop(i, axis=0)

    return tmp

In [7]:
# given (dvfs, qps, itr), concatenate all per-core logs into one big dataframe
def concat_core_logs(dvfs, qps, itr, rapl='135', run='0'):
    print('Concatenating all per-core logs with ITR-DELAY = ', itr)
    
    # here are all the log files for this dvfs & qps
    logs_dir = qps + '_qps/linux_mcd_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
    # here are all the time-management files for this dvfs & qps
    rdtsc_dir = qps + '_qps/linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    start, end = eigen_analysis.get_rdtsc(rdtsc_file)
        
    # initializing error log file
    err_dir = 'err_logs/'
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    err_file = open(err_dir + err_filename, 'w')

    # here will be stored counter-based log data from all cores
    counters_full_df = pd.DataFrame()
    # here will be stored non-counter-based log data from all cores
    non_counters_full_df = pd.DataFrame()
    
    # TODO remove fixed core-id range
    for c in range(0,16):
        file = logs_dir + 'linux.mcd.dmesg.' + run + '_' + str(c) + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
        df = pd.read_csv(file, sep = ' ', names = cols, index_col='i')
        df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
        df['timestamp'] = df['timestamp'] - df['timestamp'].min()
        df['timestamp'] = df['timestamp'] * time_unit
        df['joules'] = df['joules'] * joules_unit

        # CONCATENATING MILLISECOND-LEVEL PER-CORE DFS
        ##############################################
        # removing zero-filled log-entries
        # -> these represent interrupt occurrences at a frequency greater than per-1ms
        counters_df = df[['joules', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'timestamp']].copy()
        counters_df = counters_df[(counters_df['joules'] > 0) & (counters_df['instructions'] > 0) \
                                            & (counters_df['cycles'] > 0) & (counters_df['ref_cycles'] > 0) \
                                            & (counters_df['llc_miss'] > 0)]

        tmp = counters_df['timestamp']        
        # computing diffs of counter readings
        df_diffs = counters_df.diff().dropna().copy()
        df_diffs.columns = [f'{c}_diff' for c in df_diffs.columns]
        df_diffs = handle_neg_diffs(df_diffs, counters_df, c, err_file)
        df_diffs = df_diffs.drop(['timestamp_diff'], axis=1)
        df_diffs['timestamp'] = tmp
        
        if counters_full_df.shape[0] == 0:
            counters_full_df = df_diffs.copy()
        else:
            counters_full_df = counters_full_df.merge(df_diffs, \
                                                      left_on = 'timestamp', \
                                                      right_on = 'timestamp', \
                                                      how='outer', \
                                                      sort=True, \
                                                      suffixes=('', '_0')).fillna(0)        

        # CONCATENATING MICROSECOND-LEVEL PER-CORE DFS
        ##############################################
        non_counters_df = df[['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp']].copy()                                                          

        if non_counters_full_df.shape[0] == 0:
            non_counters_full_df = non_counters_df.copy()
        else:
            non_counters_full_df = non_counters_full_df.merge(non_counters_df, \
                                                              left_on = 'timestamp', \
                                                              right_on = 'timestamp', \
                                                              how='outer', \
                                                              sort=True, \
                                                              suffixes=('', '_0')).fillna(0)         
            
        print('CORE: ', str(c))
        print('         NON COUNTERS:  full =', non_counters_df.shape[0], \
              '  expected:', int(20 * 10**6 / int(itr)))        
        print('         COUNTERS:      full =', counters_df.shape[0], \
              '  after computing diffs =', df_diffs.shape[0]          \
             )

    # delete error log if empty
    err_file.close()
    if (os.path.getsize(err_dir + err_filename) == 0):
        os.remove(err_dir + err_filename)
        
    print()
    print('-------------------------------------------------- PARSED 16 LOGS -------------------------')
    print()
    return counters_full_df, non_counters_full_df

In [8]:
def merge_concat_logs(counters_full_df, non_counters_full_df):
    
    # creating dfs of average per-core log readings
    df_merged_counter = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', \
                                     'llc_miss_diff', 'joules_diff'])
    df_merged_non_counter = pd.DataFrame(columns=['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc'])

    for col in df_merged_counter.columns:
        df_merged_counter[col] = (counters_full_df[[col, col+'_0']].sum(axis=1))
    df_merged_counter['timestamp'] = counters_full_df['timestamp']
    for col in df_merged_non_counter.columns:
        df_merged_non_counter[col] = (non_counters_full_df[[col, col+'_0']].sum(axis=1))
    df_merged_non_counter['timestamp'] = non_counters_full_df['timestamp']
    
    return df_merged_counter, df_merged_non_counter

In [9]:
def save_merged_logs(merged_counters_df, merged_non_counters_df, dvfs, qps, itr, rapl='135', run='0', app='mcdsilo'):
    app_dir = app + '_' + run + '_rapl_' + rapl + '/'
    merged_dir = app_dir + qps + '_merged/'
    counters_df_outdir = merged_dir + qps + '_' + dvfs + '_counters_merged/'
    non_counters_df_outdir = merged_dir + qps + '_' + dvfs + '_non_counters_merged/'
    !mkdir -p $counters_df_outdir
    !mkdir -p $non_counters_df_outdir
    counters_outfile = counters_df_outdir + dvfs + '_' + qps + '_' + itr + '_counters_merged'
    non_counters_outfile = non_counters_df_outdir + dvfs + '_' + qps + '_' + itr + '_non_counters_merged'
    merged_counters_df.to_csv(counters_outfile)
    merged_non_counters_df.to_csv(non_counters_outfile)

    print('COUNTERS DIR: ')
    !ls $counters_df_outdir
    print('NON_COUNTERS DIR: ')
    !ls $non_counters_df_outdir

In [10]:
def concat_merge_core_logs(dvfs, qps, itr, rapl='135', run='0', app='mcdsilo'):
    print(f'Concatenating and merging all per-core logs with DVFS={dvfs} ITR-DELAY={itr}, QPS={qps}')
    app_dir = app + '_' + run + '_rapl_' + rapl + '/'
    logs_dir = app_dir + qps + '_qps/linux_' + app + '_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
    rdtsc_dir = app_dir + qps + '_qps/linux_' + app +'_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    rdtsc_file = rdtsc_dir + 'linux.' + app + '.rdtsc.' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    start, end = eigen_analysis.get_rdtsc(rdtsc_file)
    err_dir = app_dir + 'err_logs/'
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    err_file = open(err_dir + err_filename, 'w')

    counters_full_df = pd.DataFrame()
    non_counters_full_df = pd.DataFrame()    
    
    cores = []
    cores = !ls $logs_dir | cut -d '_' -f2 | sort | uniq
    
    for c in cores:
        df_merged_counters = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', \
                                     'llc_miss_diff', 'joules_diff', 'timestamp'])
        df_merged_non_counters = pd.DataFrame(columns=['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp'])
        file = logs_dir + 'linux.' + app + '.dmesg.' + run + '_' + str(c) + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'

        print('Uncompressing log file..')
        !gunzip -v $file'.gz'
        
        df = pd.read_csv(file, sep = ' ', names = cols, index_col='i')
        
        print('Compressing log file..')
        !gzip -v9 $file
        
        print(start, end)
        df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
        df.loc[:, 'timestamp'] = df['timestamp'] * time_unit
        df.loc[:, 'joules'] = df['joules'] * joules_unit

        counters_df = df[['joules', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'timestamp']].copy()
        counters_df = counters_df[(counters_df['joules'] > 0) & (counters_df['instructions'] > 0) \
                                            & (counters_df['cycles'] > 0) & (counters_df['ref_cycles'] > 0) \
                                            & (counters_df['llc_miss'] > 0)]
        tmp = counters_df['timestamp']        
        df_diffs = counters_df.diff().dropna().copy()
        df_diffs.columns = [f'{c}_diff' for c in df_diffs.columns]
        df_diffs = handle_neg_diffs(df_diffs, counters_df, c, err_file)
        df_diffs = df_diffs.drop(['timestamp_diff'], axis=1)
        df_diffs['timestamp'] = tmp        
        if counters_full_df.shape[0] == 0:
            counters_full_df = df_diffs.copy()
        else:
            counters_full_df = counters_full_df.merge(df_diffs, \
                                                      left_on = 'timestamp', \
                                                      right_on = 'timestamp', \
                                                      how='outer', \
                                                      sort=True, \
                                                      suffixes=('', '_0')).fillna(0) 
            for col in df_merged_counters.columns:
                if col == 'timestamp':
                    break
                else:
                    df_merged_counters[col] = (counters_full_df[[col, col+'_0']].sum(axis=1))    
            df_merged_counters['timestamp'] = counters_full_df['timestamp']
            counters_full_df = df_merged_counters.copy()

        non_counters_df = df[['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp']].copy()                                                          
        if non_counters_full_df.shape[0] == 0:
            non_counters_full_df = non_counters_df.copy()
        else:
            non_counters_full_df = non_counters_full_df.merge(non_counters_df, \
                                                              left_on = 'timestamp', \
                                                              right_on = 'timestamp', \
                                                              how='outer', \
                                                              sort=True, \
                                                              suffixes=('', '_0')).fillna(0) 
            for col in df_merged_non_counters.columns:
                if col == 'timestamp':
                    break
                else:
                    df_merged_non_counters[col] = (non_counters_full_df[[col, col+'_0']].sum(axis=1))
            df_merged_non_counters['timestamp'] = non_counters_full_df['timestamp']
            non_counters_full_df = df_merged_non_counters.copy()
                
        print()
        print('CORE: ', str(c))
        print('         COUNTERS:      full =', counters_df.shape[0], \
              '  after computing diffs =', df_diffs.shape[0])
        print('         MERGED COUNTERS: ', df_merged_counters.shape)
        print()
        print('         NON COUNTERS:  full =', non_counters_df.shape[0], \
              '  expected:', int(20 * 10**6 / int(itr)))  
        print('         MERGED NON COUNTERS: ', df_merged_non_counters.shape)
        
    err_file.close()
    if (os.path.getsize(err_dir + err_filename) == 0):
        os.remove(err_dir + err_filename)
        
    print()
    print('-------------------------------------------------- PARSED 16 LOGS -------------------------')
    print()
    return counters_full_df, non_counters_full_df, df_merged_counters, df_merged_non_counters    

In [11]:
run = '0'
rapl = '135'
dvfs = '0x1300'
app = 'mcdsilo'
logs_dir = app + '_' + run + '_rapl_' + rapl + '/'
for qps in ['200k']:
    qps_dir = logs_dir + qps + '_qps/'
    itrs = list_itrs(qps_dir + 'linux_' + app + '_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/')
    #for itr in itrs:
    for itr in ['10']:
        counters_full_df = pd.DataFrame()
        non_counters_full_df = pd.DataFrame()

        counters_full_df, non_counters_full_df, merged_counters_df, merged_non_counters_df = concat_merge_core_logs(dvfs, qps, itr, run='0', app='mcdsilo')

        counters_full_df['timestamp'] = counters_full_df['timestamp'] - counters_full_df['timestamp'].min()
        non_counters_full_df['timestamp'] = non_counters_full_df['timestamp'] - non_counters_full_df['timestamp'].min()
        merged_counters_df['timestamp'] = merged_counters_df['timestamp'] - merged_counters_df['timestamp'].min()
        merged_non_counters_df['timestamp'] = merged_non_counters_df['timestamp'] - merged_non_counters_df['timestamp'].min()

        print_err_log(dvfs, qps, itr)
        save_merged_logs(merged_counters_df, merged_non_counters_df, dvfs, qps, itr)
        print('----------------------------------------')

Concatenating and merging all per-core logs with DVFS=0x1300 ITR-DELAY=10, QPS=200k
Uncompressing log file..
mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_0_10_0x1300_135_200000.gz:	 81.6% -- replaced with mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_0_10_0x1300_135_200000
Compressing log file..
mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_0_10_0x1300_135_200000:	 81.6% -- replaced with mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_0_10_0x1300_135_200000.gz
55092971618223 55150975630005

CORE:  0
         COUNTERS:      full = 19468   after computing diffs = 19467
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 639759   expected: 2000000
         MERGED NON COUNTERS:  (0, 5)
Uncompressing log file..
mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_1_10_0x1300_135

mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_5_10_0x1300_135_200000.gz:	 81.2% -- replaced with mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_5_10_0x1300_135_200000
Compressing log file..
mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_5_10_0x1300_135_200000:	 81.2% -- replaced with mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_5_10_0x1300_135_200000.gz
55092971618223 55150975630005

CORE:  5
         COUNTERS:      full = 19287   after computing diffs = 19286
         MERGED COUNTERS:  (211764, 6)

         NON COUNTERS:  full = 528195   expected: 2000000
         MERGED NON COUNTERS:  (5761970, 5)
Uncompressing log file..
mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_200k/linux.mcdsilo.dmesg.0_6_10_0x1300_135_200000.gz:	 81.0% -- replaced with mcdsilo_0_rapl_135/200k_qps/linux_mcdsilo_dmesg_0_0x1300_135_2

In [12]:
counters_full_df

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,timestamp
0,759573.0,1627012.0,2483328.0,5136.0,0.140361,0.000000
1,990335.0,1921538.0,2932886.0,8147.0,0.140361,0.000101
2,890310.0,1906252.0,2909541.0,6757.0,0.140361,0.000107
3,829063.0,1920069.0,2930624.0,7767.0,0.125416,0.000124
4,246164.0,1090154.0,1663904.0,2743.0,0.125416,0.000129
...,...,...,...,...,...,...
288905,131916.0,342595.0,522928.0,579.0,0.164822,19.999693
288906,912644.0,1561257.0,2382988.0,2802.0,0.147925,19.999890
288907,1094100.0,1609406.0,2456445.0,2802.0,0.147925,19.999942
288908,1307704.0,1902626.0,2904002.0,3738.0,0.164822,19.999971


In [13]:
non_counters_full_df

,rx_bytes,rx_desc,tx_bytes,tx_desc,timestamp
0,66.0,1.0,0.0,0.0,0.000000
1,122.0,1.0,0.0,0.0,0.000024
2,66.0,1.0,0.0,0.0,0.000035
3,66.0,1.0,0.0,0.0,0.000051
4,155.0,1.0,0.0,0.0,0.000070
...,...,...,...,...,...
7917479,0.0,0.0,66.0,2.0,20.001153
7917480,198.0,3.0,0.0,0.0,20.001155
7917481,0.0,0.0,66.0,2.0,20.001160
7917482,0.0,0.0,66.0,2.0,20.001162


In [14]:
merged_counters_df

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,timestamp
0,759573.0,1627012.0,2483328.0,5136.0,0.140361,0.000000
1,990335.0,1921538.0,2932886.0,8147.0,0.140361,0.000101
2,890310.0,1906252.0,2909541.0,6757.0,0.140361,0.000107
3,829063.0,1920069.0,2930624.0,7767.0,0.125416,0.000124
4,246164.0,1090154.0,1663904.0,2743.0,0.125416,0.000129
...,...,...,...,...,...,...
288905,131916.0,342595.0,522928.0,579.0,0.164822,19.999693
288906,912644.0,1561257.0,2382988.0,2802.0,0.147925,19.999890
288907,1094100.0,1609406.0,2456445.0,2802.0,0.147925,19.999942
288908,1307704.0,1902626.0,2904002.0,3738.0,0.164822,19.999971


In [15]:
merged_non_counters_df

,rx_bytes,rx_desc,tx_bytes,tx_desc,timestamp
0,66.0,1.0,0.0,0.0,0.000000
1,122.0,1.0,0.0,0.0,0.000024
2,66.0,1.0,0.0,0.0,0.000035
3,66.0,1.0,0.0,0.0,0.000051
4,155.0,1.0,0.0,0.0,0.000070
...,...,...,...,...,...
7917479,0.0,0.0,66.0,2.0,20.001153
7917480,198.0,3.0,0.0,0.0,20.001155
7917481,0.0,0.0,66.0,2.0,20.001160
7917482,0.0,0.0,66.0,2.0,20.001162
